# 서브워드 토크나이저(Subword Tokenizer)
하나의 단어를 더 작은 단위의 의미있는 서브워드로 분리하는 전처리 작업을 의미한다. 예를 들어 birthplace는 birth와 place로 분리할 수 있다. 서브워드 토크나이저가 활용되는 이유는 기계가 모르는 단어를 인식하지 못하는 OOV 문제를 해결하기 위함이다. OVV(Out-Of-Vocabulary)는 기계가 아무리 학습흘 많이 하더라도 모르는 단어가 있을 수밖에 없기 때문에 발생하는 문제다. 따라서, 기계가 인식하지 못하는 단어를 더 작은 단위로 분리하는 서브워드 토크나이저를 통해 해당 문제를 완화할 수 있다.

### Byte Pair Encoding(BPE)
글자(Character) 단위에서 점차적으로 단어 집합(vocabulary)을 만들어 내는 Bottom up 방식을 사용한다. (단어집합이란, 중복을 배제한 단어들의 집합을 의미한다.)

기존 방식에서 low, lower, newest, widest라는 단어 집합이 있다면, lowest라는 단어가 등장했을 때 제대로 인식하지 못한다.

반면 BPE 방식을 활용한다면, 
+ 글자 단위 분리: l o w 와 같이 단어를 글자 단위로 분리한다
+ 단어 집합 생성: 분리한 글자로 다시 각 유니그램이 중복되지 않는 단어 집합으로 만든다. 
+ 유니그램 쌍 통합: 빈도가 가장 높은 유니그램 쌍을 통합한다. 예를 들어, (e,s)는 es로 통합할 수 있다. 이때, 빈도가 높은 유니그램 쌍을 하나로 통합하는 과정을 사용자가 임의로 지정한 횟수만큼 반복할 수 있다.

결과적으로 위에서 생성된 단어집합을 참고하여 lowest 라는 단어를 low와 est 두 단어로 인코딩하게 된다.

In [ ]:
import re, collections
from IPython.display import display, Markdown, Latex

In [ ]:
#BPE 수행 횟수
num_merges = 10

In [ ]:
#BPE에 활용할 단어
dictionary = {'l o w </w>' : 5,
         'l o w e r </w>' : 2,
         'n e w e s t </w>':6,
         'w i d e s t </w>':3
         }

In [ ]:
def get_stats(dictionary):
    # 유니그램의 pair들의 빈도수를 카운트
    pairs = collections.defaultdict(int)
    for word, freq in dictionary.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    print('현재 pair들의 빈도수 :', dict(pairs))
    return pairs

def merge_dictionary(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

bpe_codes = {}
bpe_codes_reverse = {}

for i in range(num_merges):
    display(Markdown("### Iteration {}".format(i + 1)))
    pairs = get_stats(dictionary)
    best = max(pairs, key=pairs.get)
    dictionary = merge_dictionary(best, dictionary)

    bpe_codes[best] = i
    bpe_codes_reverse[best[0] + best[1]] = best

    print("new merge: {}".format(best))
    print("dictionary: {}".format(dictionary))

### Iteration 1

현재 pair들의 빈도수 : {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 8, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('e', 's'): 9, ('s', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3}
new merge: ('e', 's')
dictionary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 6, 'w i d es t </w>': 3}


### Iteration 2

현재 pair들의 빈도수 : {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'es'): 6, ('es', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'es'): 3}
new merge: ('es', 't')
dictionary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est </w>': 6, 'w i d est </w>': 3}


### Iteration 3

...

현재 pair들의 빈도수 : {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('new', 'est</w>'): 6, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge: ('new', 'est</w>')
dictionary: {'low </w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 9

현재 pair들의 빈도수 : {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge: ('low', '</w>')
dictionary: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 10

현재 pair들의 빈도수 : {('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge: ('w', 'i')
dictionary: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}


### SentencePiece
구글에서 공개한 SnetencePeice는 BPE 알고리즘과 Unigram Language Model Tokenizer를 구현할 수 있다. BPE와 달리 사전 토큰화 작업 없이 단어 분리 토큰화를 수행하므로 언어에 종속되지 않는다는 특징이 있다.

# text-classification

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# GPU 사용 여부 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 텍스트 인코딩
text = "Daewoo Hall is too far from the main gate"
encoded_text = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

# 모델 적용
output = model(**encoded_text.to(device))
result = torch.argmax(output.logits, dim=1).cpu().detach().numpy()[0]

# 결과 출력
if result == 0:
    print("Negative")
else:
    print("Positive")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Positive
